In [2]:
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm
import numpy as np

## Explanatory analysis

In [6]:
df = pd.read_stata("assignment_1.dta", convert_dates=False)

In [3]:
df.head(7)

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix,trend
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857,2.0
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732,3.0
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331,4.0
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125,5.0
5,1,86,0.034752,0.326062,0.512324,0.439863,6.90,0.001895,2.385681,29.740980,...,3.006608,-0.051062,0.002048,-0.121867,-0.122245,0.052056,-0.015258,0.055322,0.042765,6.0
6,1,87,0.035604,0.298270,0.527596,0.436170,6.71,0.001828,2.422633,30.993681,...,3.006608,0.024198,-0.089089,0.029374,-0.008431,-0.027923,-0.036189,0.041257,-0.193899,7.0


In [4]:
df[[
    "crmrte", 
    "prbconv", 
    "prbarr", 
    "avgsen", 
    "polpc",
    "density",
    'taxpc', 
    'west', 
    'central', 
    'urban'
]]

,crmrte,prbconv,prbarr,avgsen,polpc,density,taxpc,west,central,urban
0,0.039885,0.402062,0.289696,5.61,0.001787,2.307159,25.697630,0,1,0
1,0.038345,0.433005,0.338111,5.59,0.001767,2.330254,24.874252,0,1,0
2,0.030305,0.525703,0.330449,5.80,0.001836,2.341801,26.451443,0,1,0
3,0.034726,0.604706,0.362525,6.89,0.001886,2.346420,26.842348,0,1,0
4,0.036573,0.578723,0.325395,6.55,0.001924,2.364896,28.140337,0,1,0
...,...,...,...,...,...,...,...,...,...,...
625,0.015575,0.480392,0.226667,7.77,0.001073,0.869048,18.905853,1,0,0
626,0.013662,1.410260,0.204188,10.11,0.001109,0.872024,22.704754,1,0,0
627,0.013086,0.830769,0.180556,5.96,0.001054,0.875000,24.123611,1,0,0
628,0.012874,2.250000,0.112676,7.68,0.001088,0.880952,24.981979,1,0,0


In [7]:
df = df.assign(
    log_crmrte=np.log(df["crmrte"]),
    log_prbconv = np.log(df["prbconv"]),
    log_prbarr = np.log(df["prbarr"]),
    log_avgsen = np.log(df["avgsen"]),
    log_polpc = np.log(df["polpc"]),
    log_density = np.log(df["density"])
)

log_variables = [column for column in df.columns if column.split("_")[0] == "log" and column != "log_crmrte"]
dependent_variable = ["log_crmrte"]

In [8]:
df_final = df.dropna()

# Generate dummy variables for years
dummy_vars = pd.get_dummies(df_final["year"].astype(str), prefix="year")
df_final = pd.concat([df_final, dummy_vars], axis=1)

dummy_vars



,year_82,year_83,year_84,year_85,year_86,year_87
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,0,0,0,1,0,0
5,0,0,0,0,1,0
...,...,...,...,...,...,...
625,0,1,0,0,0,0
626,0,0,1,0,0,0
627,0,0,0,1,0,0
628,0,0,0,0,1,0


- Describing columns

In [9]:
year_dummy_columns = list(dummy_vars.columns)[1:] # year_82 is removed to avoid multicollinearity
geographical_columns = ["west", "central", "urban"]

## Estimations

- Pooled OLS

In [10]:
pooled_ols = ols("log_crmrte ~" + " + ".join(log_variables+geographical_columns+year_dummy_columns), data=df_final).fit()
print(pooled_ols.summary())


                            OLS Regression Results                            
Dep. Variable:             log_crmrte   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     136.3
Date:                Mon, 13 Mar 2023   Prob (F-statistic):          6.88e-159
Time:                        20:11:04   Log-Likelihood:                -71.700
No. Observations:                 540   AIC:                             171.4
Df Residuals:                     526   BIC:                             231.5
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.7624      0.192     -9.202      

- Random Effects (with linearmodels)

In [11]:
from linearmodels.panel import RandomEffects

random_effects_df = df_final.set_index(['county', 'trend'])

exog = sm.add_constant(random_effects_df[log_variables+geographical_columns+year_dummy_columns])

re_model = RandomEffects(random_effects_df.log_crmrte, exog).\
    fit()

print(re_model.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:             log_crmrte   R-squared:                        0.5375
Estimator:              RandomEffects   R-squared (Between):              0.7721
No. Observations:                 540   R-squared (Within):               0.4043
Date:                Mon, Mar 13 2023   R-squared (Overall):              0.7428
Time:                        20:11:10   Log-likelihood                    300.00
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      47.015
Entities:                          90   P-value                           0.0000
Avg Obs:                       6.0000   Distribution:                  F(13,526)
Min Obs:                       6.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             47.015
                            

/Users/gabrieldiasmp/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


- Fixed Effects (with linearmodels)

In [12]:
# (we should test time fixed effects by calculating the joint significance of the dummy variables)

from linearmodels import PanelOLS

fixed_effects_df = df_final.set_index(['county', 'trend'])

exog = sm.add_constant(fixed_effects_df[log_variables+geographical_columns+year_dummy_columns])

fe_model = PanelOLS(fixed_effects_df.log_crmrte, exog).fit(cov_type="clustered", cluster_entity=True) #cov_type="clustered", cluster_entity=True

print(fe_model.summary)

## With statsmodels, just in case
# fixed_effects_ols = ols(
#     "log_crmrte ~" + " + ".join(log_variables+geographical_columns+year_dummy_columns), 
#     data=df_final).\
#     fit(cov_type='cluster', cov_kwds={'groups': df_final['county']})
    
# print(fixed_effects_ols.summary())

                          PanelOLS Estimation Summary                           
Dep. Variable:             log_crmrte   R-squared:                        0.7711
Estimator:                   PanelOLS   R-squared (Between):              0.8141
No. Observations:                 540   R-squared (Within):               0.2751
Date:                Mon, Mar 13 2023   R-squared (Overall):              0.7711
Time:                        20:11:27   Log-likelihood                   -71.700
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      136.32
Entities:                          90   P-value                           0.0000
Avg Obs:                       6.0000   Distribution:                  F(13,526)
Min Obs:                       6.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             67.221
                            

- Hausman test (to be tested)

In [15]:
from scipy.stats import chi2

# get the covariance matrices for the two models
cov_fixed = fe_model.cov
cov_random = re_model.cov

# calculate the difference in the estimated coefficients
b_diff = fe_model.params - re_model.params

# calculate the covariance matrix of the difference
cov_b_diff = cov_fixed - cov_random

# calculate the Hausman test statistic
hausman_stat = np.dot(b_diff.T, np.linalg.inv(cov_b_diff)).dot(b_diff)

# calculate the p-value of the test
p_value = 1 - chi2.cdf(hausman_stat, df=cov_b_diff.shape[0])

# print the results
print('Hausman test statistic: {:.4f}'.format(hausman_stat))
print('p-value: {:.4f}'.format(p_value))


Hausman test statistic: 7.6458
p-value: 0.9069
